In [ ]:
#Initialize the hardware

import matplotlib.pyplot as plt
import numpy as np

from pylablib.devices.Andor import AndorSDK2Camera, ShamrockSpectrograph
from pylablib.devices import Andor

print("SDK2 cameras:", Andor.get_cameras_number_SDK2())
print("Shamrock spectrographs:", Andor.list_shamrock_spectrographs())

#connect to the andor camera (Newton)
cam = AndorSDK2Camera(temperature=-40, fan_mode="on")

width, height = cam.get_detector_size()
print(f"Camera sensor size: {width} × {height} pixels")

# — Spectrometer —
# ShamrockSpectrograph doesn’t need a context manager
spec = ShamrockSpectrograph()
info = spec.get_device_info()  # e.g. ('SR0455', ...)
print("Spectrometer info:", info)

num_gratings = spec.get_gratings_number()
print(f"Number of gratings: {num_gratings}")

for i in range(1, num_gratings+1):
    info = spec.get_grating_info(i)
    print(f"Grating {i}: {info}")

spec.set_grating(1)  # 300 lines/mm



#use lab.drivers.motors.dll to connect and communicate with ASI stage
from stages import Stages
st = Stages()
print(st.NumberOfAxes)

SDK2 cameras: 1
Shamrock spectrographs: ['SR0455', 'SR0455']
Camera sensor size: 1600 × 400 pixels
Spectrometer info: TDeviceInfo(serial_number='SR0455')
Number of gratings: 3
Grating 1: TGratingInfo(lines=300.0, blaze_wavelength='1000', home=-15983, offset=-40)
Grating 2: TGratingInfo(lines=600.0, blaze_wavelength='500', home=144133, offset=-22)
Grating 3: TGratingInfo(lines=1200.0, blaze_wavelength='300', home=304234, offset=0)
3


In [ ]:
# ---------------------------
# Configure Spectromter and Camera
# ---------------------------
exposure_time = 0.5  # seconds
em_gain = 200        # Gain between 0-255 for EMCCD models

# Set full vertical binning (FVB)
# FVB means bin across all vertical pixels — so ROI height is 1
sensor_width, sensor_height = cam.get_detector_size()
cam.set_read_mode("fvb")
cam.set_roi(0, cam.get_detector_size()[0])
cam.set_exposure(exposure_time)
cam.set_EMCCD_gain(em_gain)

# Optional: Print some status
print(f"Exposure time: {cam.get_exposure()} s")
print(f"EM gain: {cam.get_EMCCD_gain()}")






In [ ]:
#set the min wavelength of the spectrometer
# Step 3: Override a to shift spectrum so pixel 0 ≈ 785nm
desired_lambda_min = 785  # nm
a_new = desired_lambda_min

# Step 2: Get calibration
a, b, c, d = spec.get_calibration()

# Step 4: Compute new center wavelength to pass to hardware
N = cam.get_detector_size()[0]
x_center = N // 2
lambda_center = a_new + b*x_center + c*x_center**2 + d*x_center**3

# Step 5: Set spectrometer
spec.set_wavelength(lambda_center * 1e-9)

print("Center wavelength:", spec.get_wavelength(), "nm")

def get_spectral_axis(spec, num_pixels):
    a, b, c, d = spec.get_wavelength_coefficients()
    x = np.arange(num_pixels)
    return a_new + b*x + c*x**2 + d*x**3


spec_axis = get_spectral_axis(spec, cam.get_detector_size()[0])

In [18]:
# Set wavelength to current wavelength — acts like a stop
current_wl = spec.get_wavelength()
spec.set_wavelength(current_wl)


8.000100097656251e-07

In [ ]:


frame = cam.snap()


print("Spectrum acquired. Length:", len(frame))

# ---------------------------
# Plot the spectrum
# ---------------------------
plt.figure(figsize=(8, 4))
plt.plot(frame, label='Raw EMCCD Signal')
plt.xlabel("Pixel")
plt.ylabel("Intensity (a.u.)")
plt.title("Newton EMCCD Spectrum (FVB)")
plt.legend()
plt.tight_layout()
plt.show()

In [17]:
import time
import matplotlib.pyplot as plt

# 1. Reset acquisition
cam.stop_acquisition()
cam.clear_acquisition()

# 2. Configure camera
cam.set_read_mode("fvb")
cam.set_exposure(0.5)
cam.set_EMCCD_gain(200)

# 3. Start acquisition
cam.start_acquisition()

# 4. Wait slightly longer than exposure
time.sleep(0.6)

# 5. Try to read a frame
frame = cam.read_newest_image()

# 6. Stop acquisition
cam.stop_acquisition()

# 7. Fallback if frame is None
if frame is None:
    print("⚠️ No frame from read_newest_image(). Trying read_multiple_images()")
    cam.start_acquisition()
    time.sleep(0.6)
    frames = cam.read_multiple_images(slice(0, 1))
    cam.stop_acquisition()
    frame = frames[0] if frames else None

# 8. Plot or error
if frame is None:
    print("❌ No frame acquired after all attempts.")
else:
    print("✅ Spectrum acquired. Length:", len(frame))
    plt.figure(figsize=(8, 4))
    plt.plot(frame, label="Raw EMCCD Signal")
    plt.xlabel("Pixel")
    plt.ylabel("Intensity (a.u.)")
    plt.title("Newton EMCCD Spectrum (FVB)")
    plt.legend()
    plt.tight_layout()
    plt.show()


⚠️ No frame from read_newest_image(). Trying read_multiple_images()


TypeError: 'slice' object is not iterable

In [11]:
print("Spectrum acquired. Length:", len(frame))

# ---------------------------
# Plot the spectrum
# ---------------------------
plt.figure(figsize=(8, 4))
plt.plot(frame, label='Raw EMCCD Signal')
plt.xlabel("Pixel")
plt.ylabel("Intensity (a.u.)")
plt.title("Newton EMCCD Spectrum (FVB)")
plt.legend()
plt.tight_layout()
plt.show()

TypeError: object of type 'NoneType' has no len()

In [ ]:
st._axis = 1
st.position = 0

In [19]:
cam.set_cooler(False)  # You've already done this ✅
cam.stop_acquisition()  # Just in case it's still acquiring
cam.close()

spec.close()





In [10]:
spec.close()


In [1]:
st._motor_form.Delete()
del st

NameError: name 'st' is not defined